In [1]:
# # import re
# from collections import defaultdict, Counter
# # from itertools import groupby
# # from nltk import flatten
# # from nltk.util import trigrams as get_trigrams
# from mwu_measures.preprocessing_corpus import clean_bnc_line, preprocess_bnc
# # import mwu_measures
# from mwu_measures.processing_corpus import Corpus
import mwu_measures
import mwu_measures.processing_corpus_sqlite
from mwu_measures.compute_functions import min_max_norm
from mwu_measures import compute_functions
from mwu_measures import processing_corpus
# from mwu_measures.mwu_functions import get_association, get_entropy_dif
from collections import defaultdict, Counter
# from nltk import FreqDist
import numpy as np
import pandas as pd
import json

In [2]:
this_corpus = mwu_measures.processing_corpus.process_corpus(corpus_name = 'bnc', corpus_dir='small_corpus.txt', chunk_size=1000000, verbose=True)

8864 lines processed
17792 lines processed
25147 lines processed
33162 lines processed
41755 lines processed
51542 lines processed
61364 lines processed
68925 lines processed
77269 lines processed
85997 lines processed
94339 lines processed
104655 lines processed
112340 lines processed
123640 lines processed
133269 lines processed
143097 lines processed
152333 lines processed
162336 lines processed
172733 lines processed
180833 lines processed
188553 lines processed
196165 lines processed
204099 lines processed
212111 lines processed
219877 lines processed
228642 lines processed
237401 lines processed
246111 lines processed
253930 lines processed
260878 lines processed
269188 lines processed
276338 lines processed
286268 lines processed
295632 lines processed
304704 lines processed
314564 lines processed
325720 lines processed
336644 lines processed
347508 lines processed
356459 lines processed
365035 lines processed
376796 lines processed
385311 lines processed
393768 lines processed


In [3]:
this_corpus.create_totals()
this_corpus.set_getting_functions()

In [45]:
this_corpus.corpus_conn.execute("DROP TABLE bigram_fw_query")

In [46]:
this_corpus.corpus_conn.execute("""
                        CREATE OR REPLACE TEMPORARY TABLE bigram_fw_query (query TEXT, ug_1 TEXT)
                        """)

In [47]:
this_corpus.corpus_conn.executemany("INSERT INTO bigram_fw_query VALUES (?, ?)", [['most of', 'most'], ['one of', 'one'], ['puta', 'lawea']])

In [48]:
this_corpus.corpus_conn.execute("SELECT * FROM bigram_fw_query").fetch_df()

,query,ug_1
0,most of,most
1,one of,one
2,puta,lawea


In [ ]:
this_corpus.corpus_conn.execute(
    """
    CREATE OR REPLACE TABLE filtered_db AS 
    SELECT corpus, ug_1, ug_2, SUM(freq) AS freq
    FROM trigram_db
    WHERE ug_1 IN (SELECT ug_1 FROM bigram_fw_query)
    GROUP BY corpus, ug_1, ug_2
""")
this_corpus.corpus_conn.execute(
    """
    CREATE OR REPLACE TABLE filtered_db AS                  
    SELECT query, corpus, ug_2, SUM(freq) as freq
    FROM bigram_fw_query
    INNER JOIN filtered_db
    USING(ug_1)
    GROUP BY query, corpus, ug_2
""")
x = this_corpus.corpus_conn.execute("SELECT * FROM filtered_db").fetch_df()
this_corpus.corpus_conn.execute(
    """
CREATE OR REPLACE TABLE filtered_db AS 
SELECT query, corpus, json_group_object(ug_2, freq) AS freq_dist
FROM filtered_db
WHERE ug_2 IS DISTINCT FROM NULL
GROUP BY query, corpus
""")
this_corpus.corpus_conn.execute(
    """
CREATE OR REPLACE TABLE filtered_db AS
SELECT query, json_group_object(corpus, freq_dist) as corpus_array
FROM filtered_db
GROUP BY query
""")

In [50]:
set(x['query'])

{'most of', 'one of'}

In [19]:
this_corpus.corpus_conn.execute("PRAGMA table_info(bigram_fw_query)").fetch_df()

,cid,name,type,notnull,dflt_value,pk
0,0,query,VARCHAR,False,None,False
1,1,ug_1,VARCHAR,False,None,False


In [15]:
this_json = this_corpus.corpus_conn.execute("""
    SELECT json_group_object(query, corpus_array)
    FROM filtered_db
""").fetchone()[0]

In [17]:
import orjson
orjson.loads(this_json)['most of']

{'J': {'are': 1.0,
  'areas': 2.0,
  'autobiographies': 1.0,
  'cases': 11.0,
  'courts': 1.0,
  'crucial': 1.0,
  'dedicated': 1.0,
  'experimental': 1.0,
  'extraordinary': 1.0,
  'famous': 2.0,
  'favourable': 1.0,
  'fuel': 1.0,
  'illbred': 1.0,
  'influential': 2.0,
  'irritable': 1.0,
  'likely': 8.0,
  'moulding': 1.0,
  'nearly': 1.0,
  'pertinent': 1.0,
  'polluted': 1.0,
  'popularly': 1.0,
  'poultry': 1.0,
  'reasonably': 1.0,
  'recent': 3.0,
  'regrettable': 1.0,
  'serious': 5.0,
  'skilled': 1.0,
  'teams': 1.0,
  'the': 2.0,
  'under': 1.0,
  'viable': 1.0,
  'widely': 3.0,
  'ambitious': 2.0,
  'apparent': 1.0,
  'applications': 1.0,
  'baking': 1.0,
  'centres': 1.0,
  'commercially': 1.0,
  'conventional': 1.0,
  'critics': 1.0,
  'curious': 2.0,
  'densely': 1.0,
  'environmentalist': 1.0,
  'general': 2.0,
  'immediate': 1.0,
  'instances': 1.0,
  'kids': 1.0,
  'lovely': 1.0,
  'notable': 1.0,
  'often': 1.0,
  'people': 8.0,
  'public': 1.0,
  'romantic': 1.0,


In [145]:
# this_corpus.corpus_conn.execute("""
#                 CREATE OR REPLACE TABLE filtered_db AS (
#                     SELECT corpus, ug_1, ug_2, SUM(freq) AS freq
#                     FROM trigram_db
#                     WHERE ug_1 IN (SELECT ug_1 FROM bigram_fw_query)
#                     GROUP BY corpus, ug_1, ug_2
#                 )""")
# print('Joining with query...')
x = this_corpus.corpus_conn.execute("""
                WITH filtered_db AS (
                    SELECT corpus, ug_1, ug_2, SUM(freq) AS freq
                    FROM trigram_db
                    WHERE ug_1 IN (SELECT ug_1 FROM bigram_fw_query)
                    GROUP BY corpus, ug_1, ug_2
                ), joined_db AS (
                    SELECT query, corpus, ug_2, freq
                    FROM bigram_fw_query
                    LEFT JOIN filtered_db
                    USING(ug_1)
                ), target_db AS (
                    SELECT query, corpus, json_group_object(ug_2, freq) AS freq_dist
                    FROM joined_db
                    GROUP BY query, corpus)
                SELECT query, json_group_object(corpus, freq_dist) as corpus_array
                FROM target_db
                GROUP BY query
                """).fetchall()

In [147]:
from itertools import groupby

In [ ]:
['most of']['J']

{'a': 1.0,
 'acceptable': 1.0,
 'accurately': 1.0,
 'advanced': 2.0,
 'agricultural': 1.0,
 'appropriately': 2.0,
 'aristocratic': 1.0,
 'badly': 1.0,
 'charismatic': 1.0,
 'common': 15.0,
 'controversially': 1.0,
 'dealings': 1.0,
 'definitely': 10.0,
 'development': 1.0,
 'distinctive': 1.0,
 'district': 1.0,
 'exceptional': 1.0,
 'exhibitors': 1.0,
 'explosive': 1.0,
 'financial': 1.0,
 'frequently': 1.0,
 'gorgeous': 1.0,
 'high': 1.0,
 'industry': 1.0,
 'laymen': 1.0,
 'like': 1.0,
 'longed': 1.0,
 'major': 1.0,
 'natural': 4.0,
 'needed': 1.0,
 'not': 1.0,
 'noticeably': 1.0,
 'one': 1.0,
 'outstanding': 1.0,
 'over': 1.0,
 'passionate': 1.0,
 'powerful': 1.0,
 'prevalent': 2.0,
 "professor's": 1.0,
 'rapid': 1.0,
 'renowned': 1.0,
 'restrictions': 1.0,
 'rhythmic': 1.0,
 'routine': 2.0,
 'scots': 1.0,
 'significant': 10.0,
 'sixth': 1.0,
 'sporty': 1.0,
 "tenant's": 1.0,
 'turbulent': 1.0,
 'unhappy': 1.0,
 'union': 1.0,
 'urban': 1.0,
 'visible': 1.0,
 'vividly': 1.0,
 'vulnera

In [135]:
json.loads(x['corpus_array'][0])[0][0]['J'][0]

[{'accurate': 1.0},
 {'africans': 1.0},
 {'aged': 1.0},
 {'alarming': 2.0},
 {'and': 1.0},
 {'are': 6.0},
 {'areas': 1.0},
 {'attractive': 3.0},
 {'authoritative': 2.0},
 {'british': 1.0},
 {'cases': 24.0},
 {'chemical': 1.0},
 {'complex': 3.0},
 {'courts': 1.0},
 {'crucial': 3.0},
 {'damaging': 1.0},
 {'delicate': 1.0},
 {'desirable': 2.0},
 {'drivers': 1.0},
 {'eccentric': 1.0},
 {'egyptians': 1.0},
 {'electors': 1.0},
 {'enchanting': 1.0},
 {'escape': 1.0},
 {'essential': 2.0},
 {'estate': 1.0},
 {'evenings': 1.0},
 {'experimental': 2.0},
 {'fabulous': 1.0},
 {'famous': 2.0},
 {'favourable': 1.0},
 {'fighting': 1.0},
 {'firms': 4.0},
 {'fortuitous': 1.0},
 {'girls': 1.0},
 {"government's": 1.0},
 {'industrialised': 1.0},
 {'influential': 1.0},
 {'inventive': 1.0},
 {'leases': 2.0},
 {'libel': 1.0},
 {'likely': 18.0},
 {'ofthe': 1.0},
 {'painful': 1.0},
 {'passed': 1.0},
 {'pertinent': 1.0},
 {'polluted': 4.0},
 {'potent': 1.0},
 {'poultry': 1.0},
 {'predominant': 1.0},
 {'prestigiou

In [96]:
this_corpus.corpus_conn.execute(temp_query_2).fetch_df()

,corpus,ug_1,"list(json_object(ug_2, freq))"
0,E,one,"[{""NUMBER"":4.0}, {""agent"":1.0}, {""agreed"":1.0}..."
1,K,most,"[{""absorbent"":2.0}, {""absurd"":1.0}, {""actresse..."
2,E,most,"[{""aggressive"":1.0}, {""also"":1.0}, {""apposite""..."
3,K,one,"[{""account"":2.0}, {""admits"":2.0}, {""adult"":1.0..."
4,F,one,"[{""6week"":1.0}, {""a"":22.0}, {""above"":1.0}, {""a..."
5,J,one,"[{""a"":11.0}, {""ac"":1.0}, {""actually"":2.0}, {""a..."
6,H,one,"[{""a"":27.0}, {""above"":1.0}, {""abstinent"":1.0},..."
7,B,most,"[{""NUMBER"":1.0}, {""abstract"":1.0}, {""active"":2..."
8,D,most,"[{""people"":1.0}, {""up"":1.0}, {""influential"":1...."
9,G,most,"[{""absolute"":1.0}, {""afternoons"":1.0}, {""aggre..."


In [ ]:
%timeit this_corpus.corpus_conn.execute(temp_query).fetch_df()

28.9 ms ± 569 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


,query,corpus,ug_1,ug_2,freq
0,one of,H,one,of,2780.0
1,one of,C,one,of,2710.0
2,one of,A,one,of,2521.0
3,one of,K,one,of,2091.0
4,one of,G,one,of,1255.0
...,...,...,...,...,...
21494,most of,C,most,wellknown,1.0
21495,most of,C,most,which,1.0
21496,most of,C,most,who,1.0
21497,most of,C,most,women's,1.0


,corpus,ug_1,ug_2,freq
0,B,one,accurate,1.0
1,B,one,activity,2.0
2,B,one,afternoon,4.0
3,B,one,american,4.0
4,B,one,and,58.0
...,...,...,...,...
21494,H,one,world,4.0
21495,H,one,worth,1.0
21496,H,one,wouldbe,3.0
21497,H,one,year,71.0


NameError: name 'y' is not defined

In [ ]:
this_corpus.corpus_conn.execute("""
                                SELECT query.query, query.ug_1, data.ug_2, SUM(data.freq) as freq
                                FROM bigram_fw_query AS query
                                LEFT JOIN (
                                    SELECT corpus, ug_1, ug_2, freq
                                    FROM trigram_db
                                    WHERE ug_1 IN query.ug_1
                                ) AS data
                                ON query.ug_1 = data.ug_1
                                GROUP BY query, corpus, query.ug_1, ug_2
                                """).fetch_df()

,query,ug_1,ug_2,freq
0,most of,most,aggressive,1.0
1,most of,most,apologetic,1.0
2,most of,most,appropriate,2.0
3,most of,most,beneficial,1.0
4,most of,most,breaches,1.0
...,...,...,...,...
21494,one of,one,wonder,1.0
21495,one of,one,wood,1.0
21496,one of,one,worker,1.0
21497,one of,one,workers,1.0


In [23]:
this_corpus.corpus_conn.execute("""SELECT bigram_fw_query.query, trigram_db.corpus, bigram_fw_query.ug_1, trigram_db.ug_2, SUM(freq) as freq
                                FROM bigram_fw_query
                                LEFT JOIN trigram_db
                                ON bigram_fw_query.ug_1 = trigram_db.ug_1
                                GROUP BY query, corpus, bigram_fw_query.ug_1, ug_2
                                ORDER BY freq DESC
                                """).fetchdf()

,query,corpus,ug_1,ug_2,freq
0,one of,H,one,of,2780.0
1,one of,C,one,of,2710.0
2,one of,A,one,of,2521.0
3,one of,K,one,of,2091.0
4,one of,G,one,of,1255.0
...,...,...,...,...,...
21494,most of,C,most,wellknown,1.0
21495,most of,C,most,which,1.0
21496,most of,C,most,who,1.0
21497,most of,C,most,women's,1.0


In [19]:
x

[('one of', 'H', 'one', 'of', 2780),
 ('one of', 'C', 'one', 'of', 2710),
 ('one of', 'A', 'one', 'of', 2521),
 ('one of', 'K', 'one', 'of', 2091),
 ('one of', 'G', 'one', 'of', 1255),
 ('one of', 'F', 'one', 'of', 1240),
 ('one of', 'B', 'one', 'of', 1133),
 ('one of', 'E', 'one', 'of', 1057),
 ('one of', 'J', 'one', 'of', 865),
 ('most of', 'C', 'most', 'of', 807),
 ('most of', 'A', 'most', 'of', 756),
 ('most of', 'H', 'most', 'of', 747),
 ('most of', 'K', 'most', 'of', 422),
 ('most of', 'F', 'most', 'of', 394),
 ('most of', 'B', 'most', 'of', 349),
 ('most of', 'G', 'most', 'of', 345),
 ('most of', 'E', 'most', 'of', 312),
 ('one of', 'H', 'one', 'or', 269),
 ('one of', 'K', 'one', 'in', 264),
 ('one of', 'K', 'one', 'and', 251),
 ('one of', 'C', 'one', 'in', 247),
 ('most of', 'J', 'most', 'of', 232),
 ('one of', 'K', 'one', 'two', 220),
 ('one of', 'C', 'one', 'or', 211),
 ('most of', 'H', 'most', 'important', 210),
 ('one of', 'H', 'one', 'in', 201),
 ('most of', 'C', 'most', '

In [17]:
this_corpus.corpus_conn.execute("SELECT * FROM bigram_fw_query").fetch_df()

,query,ug_1
0,"[most of, most]","[one of, one]"


In [4]:
this_corpus.corpus_conn.executemany(this_corpus.fw_trigram_query, [['most of the', 'most', 'of'], ['one of the', 'one', 'of']])

In [5]:
from itertools import groupby

trigrams_fw = this_corpus.corpus_conn.execute("SELECT * FROM trigram_fw_query ORDER BY query, corpus, freq DESC").fetchall()
trigrams_fw = {ngram: {corpus: Counter({ngram_info[2]: ngram_info[3] for ngram_info in corpus_info}) for corpus, corpus_info in groupby(data, lambda x: x[1])} for ngram, data in groupby(trigrams_fw, lambda x: x[0])}

In [6]:
trigrams_fw

{'most of the': {'A': Counter({'the': 53,
           'his': 13,
           'them': 13,
           'us': 9,
           'these': 6,
           'which': 6,
           'it': 4,
           'what': 3,
           'whom': 3,
           'her': 2,
           'their': 2,
           'those': 2,
           'a': 1,
           'austria': 1,
           'containers': 1,
           'its': 1,
           'my': 1,
           'our': 1,
           'radio': 1,
           'shostakovich': 1,
           'that': 1,
           'this': 1,
           "today's": 1,
           'west': 1,
           'westminster': 1,
           'yesterday': 1}),
  'B': Counter({'the': 45,
           'them': 6,
           'its': 4,
           'us': 4,
           'her': 2,
           'these': 2,
           'those': 2,
           'a': 1,
           'all': 1,
           'an': 1,
           "britain's": 1,
           'each': 1,
           'his': 1,
           'it': 1,
           'my': 1,
           "nigeria's": 1,
           'our': 1,
     

In [ ]:
            # distribution = {corpus: Counter({v[2]: v[3] for v in vs}) for corpus, vs in groupby(queried_ngram, lambda x: x[0])}

x = this_corpus.corpus_conn.execute("SELECT * FROM bigram_query ORDER BY query, corpus, freq DESC").fetchall()
x = {ngram: {corpus: Counter({ngram_info[2]: ngram_info[3] for ngram_info in corpus_info}) for corpus, corpus_info in groupby(data, lambda x: x[1])} for ngram, data in groupby(x, lambda x: x[0])}

{'most of': {'A': Counter({'of': 130,
           'important': 42,
           'people': 20,
           'popular': 13,
           'recent': 10,
           'basic': 10,
           'successful': 9,
           'powerful': 9,
           'difficult': 8,
           'interesting': 8,
           'common': 8,
           'significant': 8,
           'effective': 7,
           'importantly': 6,
           'likely': 6,
           'part': 6,
           'beautiful': 5,
           'famous': 5,
           'serious': 5,
           'striking': 5,
           'modern': 4,
           'other': 4,
           'remarkable': 4,
           'suitable': 4,
           'natural': 4,
           'heinous': 3,
           'impressive': 3,
           'obvious': 3,
           'probably': 3,
           'spectacular': 3,
           'widely': 3,
           'dangerous': 3,
           'vital': 3,
           'in': 3,
           'reliable': 3,
           'strongly': 3,
           'conservative': 3,
           'days': 3,
          

In [ ]:
# distribution = distribution.groupby(by='corpus')[['ug_3', 'freq']].apply(lambda x: Counter(dict(zip(x['ug_3'], x['freq']))))
# distribution = distribution.to_dict()
# distribution = Counter(dict(zip(distribution.ug_3, distribution.freq)))

In [8]:
x = this_corpus.get_bw_distribution('most of the')
# {data[0]: Counter({data[2]: data[3]}) for data in x}
x

{'A': Counter({('one', 'of'): 298,
          ('part', 'of'): 116,
          ('some', 'of'): 116,
          ('end', 'of'): 112,
          ('out', 'of'): 112,
          ('it', 'was'): 67,
          ('because', 'of'): 60,
          ('back', 'to'): 54,
          ('in', 'which'): 54,
          ('it', 'is'): 53,
          ('most', 'of'): 53,
          ('to', 'be'): 52,
          ('side', 'of'): 51,
          ('and', 'in'): 48,
          ('many', 'of'): 47,
          ('members', 'of'): 46,
          ('in', 'NUMBER'): 43,
          ('up', 'to'): 41,
          ('much', 'of'): 39,
          ('rest', 'of'): 39,
          ('cent', 'of'): 38,
          ('is', 'that'): 38,
          ('on', 'to'): 37,
          ('away', 'from'): 35,
          ('member', 'of'): 35,
          ('this', 'is'): 33,
          ('NUMBER', 'and'): 32,
          ('of', 'all'): 32,
          ('such', 'as'): 31,
          ('to', 'see'): 31,
          ('NUMBER', 'of'): 30,
          ('parts', 'of'): 30,
          ('top', 'of'): 3

In [42]:
from itertools import groupby
{corpus: Counter({v[1]: v[3] for v in vs}) for corpus, vs in groupby(x, lambda x: x[0])}

{'E': Counter({'concept': 24,
          'circumstances': 9,
          'conception': 9,
          'conference': 8,
          'ahead': 7,
          'body': 7,
          'actions': 5,
          'advent': 5,
          'chancellor': 5,
          'conduct': 5,
          'allocation': 4,
          'appeal': 3,
          'assortment': 3,
          'authorities': 3,
          'capital': 3,
          'coincidence': 3,
          'conditions': 3,
          'achievement': 2,
          'arrangement': 2,
          'beginnings': 2,
          'bill': 2,
          'brink': 2,
          'ceiling': 2,
          'citizens': 2,
          'comfort': 2,
          'comté': 2,
          'conclusion': 2,
          'confiscation': 2,
          '10–14': 1,
          '20%': 1,
          '40%': 1,
          'adoption': 1,
          'adverbials': 1,
          'aerosol': 1,
          'affiliations': 1,
          'aficionados': 1,
          'allegiance': 1,
          'apparatus': 1,
          'appliqués': 1,
          

In [15]:
this_corpus.get_bw_distribution("oily rag")

{'A': Counter({'of': 1, 'red': 1}),
 'B': Counter({'leaf': 1, 'of': 1, 'shmatte': 1}),
 'C': Counter({'a': 1, 'by': 1, 'nymph': 1, 'red': 1, 'the': 1, 'to': 1}),
 'E': Counter({'of': 1}),
 'F': Counter({'ridiculous': 1}),
 'H': Counter({'a': 2,
          'lighted': 1,
          'local': 1,
          'my': 1,
          'oily': 1,
          'that': 1,
          'the': 1}),
 'K': Counter({'the': 2, 'a': 1, 'and': 1, 'me': 1})}

In [ ]:
get_scratch = this_corpus.corpus_conn.prepare("""
                                 PREPARE get_fw_bigram AS
                                 SELECT corpus, ug_1, ug_2, SUM(freq) AS freq 
                                 FROM trigram_db WHERE ug_1 = $1 
                                 GROUP BY corpus, ug_1, ug_2
                                 """)

In [ ]:
this_corpus.get_fw_distribution("most of")

In [ ]:
mwu_examples = pd.read_csv('MultiwordExpression_Concreteness_Ratings.csv')
mwu_examples['length'] = mwu_examples['Expression'].apply(lambda x: len(x.split()))
mwu_examples = mwu_examples.loc[(mwu_examples['length'] == 2) | (mwu_examples['length'] == 3)]
mwu_examples['Expression'] = mwu_examples['Expression'].apply(lambda x: x.lower())
mwu_examples = mwu_examples.sample(10000)

In [ ]:
ngram_list = mwu_examples.Expression
bigrams = [ngram.split() for ngram in ngram_list if len(ngram.split()) == 2]
trigrams = [ngram.split() for ngram in ngram_list if len(ngram.split()) == 3]

In [ ]:
trigram_query = pd.DataFrame(trigrams, columns=['ug_1', 'ug_2', 'ug_3'])
this_corpus.corpus_conn.execute("CREATE TEMPORARY TABLE trigram_query (ug_1 TEXT, ug_2 TEXT, ug_3 TEXT)")
this_corpus.corpus_conn.register('trigram_query_list', trigram_query)
this_corpus.corpus_conn.execute("INSERT INTO trigram_query SELECT * FROM trigram_query_list")

In [ ]:
trigram_per_corpus = this_corpus.corpus_conn.execute("""
    SELECT * FROM trigram_db 
    WHERE (ug_1 IN (SELECT ug_1 FROM trigram_query) AND ug_2 IN (SELECT ug_2 FROM trigram_query)) OR (ug_3 IN (SELECT ug_3 FROM trigram_query))
""").fetch_df()
# trigram_total = this_corpus.corpus_conn.execute("""
#     SELECT ug_1, ug_2, ug_3, sum(freq) as freq FROM trigram_db 
#     WHERE (ug_1 IN (SELECT ug_1 FROM trigram_query) AND ug_2 IN (SELECT ug_2 FROM trigram_query)) OR (ug_3 IN (SELECT ug_3 FROM trigram_query))
#     GROUP BY (ug_1, ug_2, ug_3)
# """).fetch_df()

In [ ]:
bigram_query = pd.DataFrame(bigrams, columns=['ug_1', 'ug_2'])
this_corpus.corpus_conn.execute("CREATE TEMPORARY TABLE bigram_query (ug_1 TEXT, ug_2 TEXT)")
this_corpus.corpus_conn.register('bigram_query_list', bigram_query)
this_corpus.corpus_conn.execute("INSERT INTO bigram_query SELECT * FROM bigram_query_list")

In [ ]:
# this_corpus.corpus_conn.execute("DROP TABLE bigram_query")

In [ ]:
bigram_per_corpus = this_corpus.corpus_conn.execute("""
    SELECT corpus, ug_1, ug_2, sum(freq) AS freq FROM trigram_db 
    WHERE (ug_1 IN (SELECT ug_1 FROM bigram_query)) OR (ug_2 IN (SELECT ug_2 FROM bigram_query))
    GROUP BY corpus, ug_1, ug_2
""").fetch_df()
# bigram_total = this_corpus.corpus_conn.execute("""
#     SELECT ug_1, ug_2, sum(freq) AS freq FROM trigram_db 
#     WHERE (ug_1 IN (SELECT ug_1 FROM bigram_query)) OR (ug_2 IN (SELECT ug_2 FROM bigram_query))
#     GROUP BY ug_1, ug_2
# """).fetch_df()

In [ ]:
# takes too long, filter DF first

fw_dists = {ngram_1: {corpus:  Counter(dict(zip(data['ug_2'], data['freq']))) for corpus, data in group.groupby('corpus')} for ngram_1, group in bigram_per_corpus.groupby('ug_1')}

In [ ]:
fw_dists['A']

In [ ]:
this_corpus.corpus_conn.execute("SELECT * FROM trigram_db ORDER BY corpus").fetch_df()

In [ ]:
this_corpus = mwu_measures.processing_corpus.process_corpus(corpus_name='bnc', corpus_dir='small_corpus.txt', verbose=True, chunk_size=1000000)
this_corpus.create_totals()